In [32]:
import torch
import os
from omegaconf import OmegaConf, open_dict
from pytorch_lightning import Trainer

import nemo.collections.asr as nemo_asr
from pathlib import Path


In [33]:
# Load the model from a local .nemo file

MODEL_ROOT = "/external2/models/hf/stt_en_conformer_ctc_large/"
MODEL_ROOT = Path(MODEL_ROOT)
model_path = MODEL_ROOT / "stt_en_conformer_ctc_large.nemo"

cfg = nemo_asr.models.ASRModel.restore_from(restore_path=model_path, return_config=True)

# If you need to get the configuration, you can access it from the loaded model

print(cfg)

{'sample_rate': 16000, 'log_prediction': True, 'ctc_reduction': 'mean_batch', 'train_ds': {'manifest_filepath': '/data/NeMo_ASR_SET/English/v2.0/train/tarred_audio_manifest.json', 'sample_rate': 16000, 'batch_size': 32, 'shuffle': True, 'num_workers': 8, 'pin_memory': True, 'use_start_end_token': False, 'trim_silence': False, 'max_duration': 20.0, 'min_duration': 0.1, 'shuffle_n': 2048, 'is_tarred': True, 'tarred_audio_filepaths': '/data/NeMo_ASR_SET/English/v2.0/train/audio__OP_0..4095_CL_.tar'}, 'validation_ds': {'manifest_filepath': ['/data/ASR/LibriSpeech/librispeech_withsp2/manifests/librivox-dev-other.json', '/data/ASR/LibriSpeech/librispeech_withsp2/manifests/librivox-dev-clean.json', '/data/ASR/LibriSpeech/librispeech_withsp2/manifests/librivox-test-other.json', '/data/ASR/LibriSpeech/librispeech_withsp2/manifests/librivox-test-clean.json'], 'sample_rate': 16000, 'batch_size': 16, 'shuffle': False, 'num_workers': 8, 'pin_memory': True, 'use_start_end_token': False, 'is_tarred':

In [34]:
from nemo.core import adapter_mixins

# Utility method to check and update the model config
def update_model_config_to_support_adapter(model_cfg):
    with open_dict(model_cfg):
        adapter_metadata = adapter_mixins.get_registered_adapter(model_cfg.encoder._target_)
        if adapter_metadata is not None:
            model_cfg.encoder._target_ = adapter_metadata.adapter_class_path

    print("Updated encoder _target_ model :", model_cfg.encoder._target_)
    return model_cfg

In [35]:
cfg = update_model_config_to_support_adapter(cfg)

Updated encoder _target_ model : nemo.collections.asr.modules.conformer_encoder.ConformerEncoderAdapter


In [36]:
model = nemo_asr.models.ASRModel.restore_from(model_path, override_config_path=cfg)

[NeMo I 2024-07-16 06:21:17 mixins:172] Tokenizer SentencePieceTokenizer initialized with 128 tokens


[NeMo W 2024-07-16 06:21:17 modelPT:165] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data/NeMo_ASR_SET/English/v2.0/train/tarred_audio_manifest.json
    sample_rate: 16000
    batch_size: 32
    shuffle: true
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 20.0
    min_duration: 0.1
    shuffle_n: 2048
    is_tarred: true
    tarred_audio_filepaths: /data/NeMo_ASR_SET/English/v2.0/train/audio__OP_0..4095_CL_.tar
    
[NeMo W 2024-07-16 06:21:17 modelPT:172] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath:
    - /data/ASR/LibriSpeech/librispeech_withs

[NeMo I 2024-07-16 06:21:17 features:289] PADDING: 0
[NeMo I 2024-07-16 06:21:19 save_restore_connector:249] Model EncDecCTCModelBPE was successfully restored from /external2/models/hf/stt_en_conformer_ctc_large/stt_en_conformer_ctc_large.nemo.


In [37]:
data_dir = "/home/ksingla/workspace/PromptingNemo/data/synthetic/"
TRAIN_MANIFEST = os.path.join(data_dir, "text_train_cleaned_manifest_v3.json")
TEST_MANIFEST = os.path.join(data_dir, "text_valid_cleaned_manifest_v4.json")
ALL_TAGS = os.path.join(data_dir, "alltags_v4.txt")
taglist = []
with open(ALL_TAGS, 'r') as f:
    for line in f:
        word, tag = line.split()
        taglist.append(tag)

In [38]:
import subprocess
import os
import sentencepiece as spm
import logging
import sys
import json

def generate_sentencepiece_model_pb2(script_dir, proto_file_path):
    # Construct the command
    command = [
        'protoc',
        f'--python_out={script_dir}',
        proto_file_path
    ]
    
    try:
        # Run the command
        subprocess.run(command, check=True)
        print("Successfully generated sentencepiece_model_pb2.py")
    except subprocess.CalledProcessError as e:
        print(f"Error generating sentencepiece_model_pb2.py: {e}")
        sys.exit(1)

def edit_spt_model(input_file, output_folder, tokens, vocab_file, vocab_txt_file, is_userdefined=False):
    from sentencepiece_model_pb2 import ModelProto  # Ensure this import is after the proto generation
    
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    output_model_file = os.path.join(output_folder, 'tokenizer.model')
    output_vocab_file = os.path.join(output_folder, 'tokenizer.vocab')
    output_vocab_txt_file = os.path.join(output_folder, 'vocab.txt')

    token_type = 3
    if is_userdefined:
        token_type = 4

    model = ModelProto()
    model.ParseFromString(open(input_file, 'rb').read())

    existing_tokens = {piece.piece for piece in model.pieces}

    new_tokens = []
    for token in tokens:
        if token in existing_tokens:
            logging.warning(f"Special Token '{token}' already exists in the input model, skipping.")
            continue
        piece = model.SentencePiece(piece=token, score=0.0, type=token_type)
        model.pieces.append(piece)
        new_tokens.append(token)

    sp = spm.SentencePieceProcessor()
    try:
        sp.LoadFromSerializedProto(model.SerializeToString())
        for token in new_tokens:
            id = sp.piece_to_id(token)
            logging.info(f"Created token '{token}' at ID {id}")
        logging.info(f"New tokenizer vocab size: {sp.get_piece_size()}")
    except:
        logging.error("Could not appropriately configure new tokenizer. Verify if the special tokens already exist.")
        sys.exit(1)

    with open(output_model_file, 'wb') as outf:
        outf.write(model.SerializeToString())

    logging.info(f"Created new tokenizer at: {output_model_file}")

    # Read the original vocab file and append the new tokens
    with open(vocab_file, 'r') as original_vocab_file:
        original_vocab = original_vocab_file.readlines()

    with open(output_vocab_file, 'w') as updated_vocab_file:
        updated_vocab_file.writelines(original_vocab)
        for token in new_tokens:
            updated_vocab_file.write(f"{token}\n")

    # Update vocab.txt
    with open(vocab_txt_file, 'r') as original_vocab_txt_file:
        original_vocab_txt = original_vocab_txt_file.readlines()

    with open(output_vocab_txt_file, 'w') as updated_vocab_txt_file:
        updated_vocab_txt_file.writelines(original_vocab_txt)
        for token in new_tokens:
            updated_vocab_txt_file.write(f"{token}\n")

    logging.info(f"Updated vocab files: {output_vocab_file}, {output_vocab_txt_file}")

def update_model_config(model, new_model_path):
    model['cfg']['tokenizer']['model_path'] = new_model_path
    logging.info(f"Updated model configuration with new tokenizer model path: {new_model_path}")


# Define input and output paths
input_folder = MODEL_ROOT / "tokenizer"
output_folder = MODEL_ROOT / "new_tokenizer"


#input_folder = '/external/ksingla/models/nemo/stt_en_conformer_ctc_small/tokenizer'
#output_folder = '/external/ksingla/models/nemo/stt_en_conformer_ctc_small/new_tokenizer'
#proto_dir = '/path/to/save/proto'  # Define the actual path where the proto file should be saved
#proto_file = '/path/to/sentencepiece_model.proto'  # Define the actual path to the sentencepiece_model.proto file

input_file = input_folder / 'tokenizer.model'
vocab_file = input_folder / 'tokenizer.vocab'
vocab_txt_file = input_folder / 'vocab.txt'

# input_file = os.path.join(input_folder, 'tokenizer.model')
# vocab_file = os.path.join(input_folder, 'tokenizer.vocab')
# vocab_txt_file = os.path.join(input_folder, 'vocab.txt')

# Include all single-digit integers in the tokens list
punctuations = ['.', ',', '?', '!', ';', ':', '-', '(', ')', '[', ']', '{', '}', '<', '>', '/', '\\', '|', '@', '#', '$', '%', '^', '&', '*', '+', '=', '~', '`', '_', '"', "'"]
tokens = taglist + [str(i) for i in range(10)] + punctuations
is_userdefined = True

# Step 1: Generate the sentencepiece_model_pb2.py file
#generate_sentencepiece_model_pb2(proto_dir, proto_file)

# Step 2: Edit the SentencePiece model
edit_spt_model(input_file, output_folder, tokens, vocab_file, vocab_txt_file, is_userdefined)

# Step 3: Load the model configuration and update it
# model_config_file = '/path/to/model/config.json'  # Define the actual path to the model config file
# with open(model_config_file, 'r') as f:
#     model = json.load(f)

# new_model_path = os.path.join(output_folder, 'tokenizer.model')
# update_model_config(model, new_model_path)

# # Save the updated model configuration
# with open(model_config_file, 'w') as f:
#     json.dump(model, f, indent=4)

#logging.info(f"Updated model configuration saved to: {model_config_file}")


Special Token '-' already exists in the input model, skipping.
Special Token ''' already exists in the input model, skipping.


In [39]:
model.change_vocabulary(output_folder, "bpe")

[NeMo W 2024-07-16 06:21:19 modelPT:258] You tried to register an artifact under config key=tokenizer.model_path but an artifact for it has already been registered.
[NeMo W 2024-07-16 06:21:19 modelPT:258] You tried to register an artifact under config key=tokenizer.vocab_path but an artifact for it has already been registered.
[NeMo W 2024-07-16 06:21:19 modelPT:258] You tried to register an artifact under config key=tokenizer.spe_tokenizer_vocab but an artifact for it has already been registered.


[NeMo I 2024-07-16 06:21:19 mixins:172] Tokenizer SentencePieceTokenizer initialized with 1206 tokens
[NeMo I 2024-07-16 06:21:20 ctc_bpe_models:248] 
    Replacing old number of classes (128) with new number of classes - 1206
[NeMo I 2024-07-16 06:21:20 ctc_bpe_models:290] Changed tokenizer to ['<unk>', 's', '▁', 'e', 't', 'u', 'd', 'a', 'o', 'n', 'i', '▁the', '▁a', 'm', 'y', 'l', 'h', 'p', 're', '▁s', 'g', 'r', '▁to', '▁i', 'ing', '▁and', 'f', '▁p', 'an', 'c', 'w', 'er', 'ed', '▁of', '▁in', 'k', "'", '▁w', 'ar', 'or', '▁f', 'b', '▁b', 'en', '▁you', 'al', 'le', 'in', 'll', '▁that', '▁he', 'ro', '▁t', 'es', '▁it', '▁be', 've', 'v', 'ly', '▁c', 'th', '▁o', 'ent', 'ch', 'ur', '▁we', '▁re', '▁n', 'it', '▁so', '▁co', '▁g', '▁on', '▁for', 'on', 'ce', 'ri', '▁do', '▁is', '▁ha', '▁ma', 'ver', 'li', 'ra', '▁was', 'ic', 'la', '▁e', 'se', 'ter', 'ct', 'ion', '▁ca', '▁st', '▁me', 'ir', '▁mo', '▁with', '▁but', '▁have', '▁go', '▁de', '▁ho', '▁di', '▁not', '▁know', '▁lo', '▁this', 'ation', 'ther', '

In [40]:
accelerator = 'gpu' if torch.cuda.is_available() else 'cpu'
max_steps = 30000

trainer = Trainer(devices=1, accelerator=accelerator, max_steps=max_steps,
                  enable_checkpointing=False, logger=False,
                  log_every_n_steps=50, check_val_every_n_epoch=1)

model.set_trainer(trainer)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [41]:
# utility method
import json
from nemo.collections.asr.parts.utils.manifest_utils import read_manifest

In [42]:
with open_dict(model.cfg):
  # Train Dataloader
  model.cfg.train_ds.manifest_filepath = TRAIN_MANIFEST
  model.cfg.train_ds.batch_size = 16
  model.cfg.train_ds.is_tarred = False
  model.cfg.train_ds.tarred_audio_filepaths = None

  model.cfg.validation_ds.manifest_filepath = TEST_MANIFEST
  model.cfg.validation_ds.batch_size = 16

model.setup_training_data(model.cfg.train_ds)
model.setup_multiple_validation_data(model.cfg.validation_ds)
model.setup_multiple_test_data(model.cfg.validation_ds)

[NeMo I 2024-07-16 06:21:21 collections:196] Dataset loaded with 32743 files totalling 38.38 hours
[NeMo I 2024-07-16 06:21:21 collections:197] 2 files were filtered totalling 0.01 hours
[NeMo I 2024-07-16 06:21:22 collections:196] Dataset loaded with 1650 files totalling 1.87 hours
[NeMo I 2024-07-16 06:21:22 collections:197] 0 files were filtered totalling 0.00 hours
[NeMo I 2024-07-16 06:21:22 collections:196] Dataset loaded with 1650 files totalling 1.87 hours
[NeMo I 2024-07-16 06:21:22 collections:197] 0 files were filtered totalling 0.00 hours


In [43]:
with open_dict(model.cfg):
  # Spec Augment
  model.cfg.spec_augment.freq_masks = model.cfg.spec_augment.freq_masks  # Can be changed
  model.cfg.spec_augment.freq_width = model.cfg.spec_augment.freq_width  # Can be changed
  model.cfg.spec_augment.time_masks = model.cfg.spec_augment.time_masks  # Can be changed
  model.cfg.spec_augment.time_width = model.cfg.spec_augment.time_width  # Can be changed

model.spec_augmentation = model.from_config_dict(model.cfg.spec_augment)

In [44]:
if 'optim' in model.cfg:
  print(OmegaConf.to_yaml(model.cfg.optim))

name: adamw
lr: 2.0
betas:
- 0.9
- 0.98
weight_decay: 0.001
sched:
  name: NoamAnnealing
  d_model: 512
  warmup_steps: 10000
  warmup_ratio: null
  min_lr: 1.0e-06



In [45]:
with open_dict(model.cfg):
  model.cfg.optim.lr = 0.1
  model.cfg.optim.weight_decay = 0.0
  model.cfg.optim.sched.warmup_steps = 100

model.setup_optimization(model.cfg.optim);

[NeMo I 2024-07-16 06:21:22 modelPT:723] Optimizer config = AdamW (
    Parameter Group 0
        amsgrad: False
        betas: [0.9, 0.98]
        capturable: False
        differentiable: False
        eps: 1e-08
        foreach: None
        fused: None
        lr: 0.1
        maximize: False
        weight_decay: 0.0
    )
[NeMo I 2024-07-16 06:21:22 lr_scheduler:915] Scheduler "<nemo.core.optim.lr_scheduler.NoamAnnealing object at 0x7c3af0cafa00>" 
    will be used during training (effective maximum steps = 30000) - 
    Parameters : 
    (d_model: 512
    warmup_steps: 100
    warmup_ratio: null
    min_lr: 1.0e-06
    max_steps: 30000
    )


In [46]:
if hasattr(model, 'adapter_module_names'):
  print(model.adapter_module_names)

['', 'encoder', 'decoder', 'joint']


In [47]:
for module in model.children():
  if hasattr(module, 'get_accepted_adapter_types'):
    types = module.get_accepted_adapter_types()
    print("Module : ", module.__class__.__name__)

    for tp in types:
      print(tp)
    print()

Module :  ConformerEncoderAdapter
<class 'nemo.collections.asr.parts.submodules.adapters.multi_head_attention_adapter_module.MultiHeadAttentionAdapter'>
<class 'nemo.collections.common.parts.adapter_modules.LinearAdapter'>
<class 'nemo.collections.asr.parts.submodules.adapters.multi_head_attention_adapter_module.RelPositionMultiHeadAttentionAdapter'>

Module :  ConvASRDecoder
<class 'nemo.collections.common.parts.adapter_modules.LinearAdapter'>



In [48]:
from nemo.collections.common.parts.adapter_modules import LinearAdapterConfig

In [49]:
#%% [code]
#@title Adapter Setup { display-mode: "form" }
adapter_name = "AN4" #@param {type:"string"}
adapter_dim = 64 #@param {type:"integer"}
adapter_activation = "swish" #@param {type:"string"}
adapter_norm_position = "pre" #@param ["pre", "post"]

In [50]:
adapter_cfg = LinearAdapterConfig(
    in_features=model.cfg.encoder.d_model,  # conformer specific model dim. Every layer emits this dim at its output.
    dim=adapter_dim,  # the bottleneck dimension of the adapter
    activation=adapter_activation,  # activation used in bottleneck block
    norm_position=adapter_norm_position,  # whether to use LayerNorm at the beginning or the end of the adapter
)
print(adapter_cfg)

LinearAdapterConfig(in_features=512, dim=64, activation='swish', norm_position='pre', dropout=0.0, adapter_strategy=ResidualAddAdapterStrategyConfig(stochastic_depth=0.0, l2_lambda=0.0, _target_='nemo.core.classes.mixins.adapter_mixin_strategies.ResidualAddAdapterStrategy'), _target_='nemo.collections.common.parts.adapter_modules.LinearAdapter')


In [51]:
model.summarize()

  | Name              | Type                              | Params
------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0     
1 | encoder           | ConformerEncoderAdapter           | 121 M 
2 | spec_augmentation | SpectrogramAugmentation           | 0     
3 | wer               | WER                               | 0     
4 | decoder           | ConvASRDecoder                    | 619 K 
5 | loss              | CTCLoss                           | 0     
------------------------------------------------------------------------
122 M     Trainable params
0         Non-trainable params
122 M     Total params
488.217   Total estimated model params size (MB)

In [52]:
model.add_adapter(name=adapter_name, cfg=adapter_cfg)

In [53]:
model.summarize()

  | Name              | Type                              | Params
------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0     
1 | encoder           | ConformerEncoderAdapter           | 122 M 
2 | spec_augmentation | SpectrogramAugmentation           | 0     
3 | wer               | WER                               | 0     
4 | decoder           | ConvASRDecoder                    | 619 K 
5 | loss              | CTCLoss                           | 0     
------------------------------------------------------------------------
123 M     Trainable params
0         Non-trainable params
123 M     Total params
493.010   Total estimated model params size (MB)

In [54]:
model.set_enabled_adapters(enabled=False)  # disable all adapters
model.set_enabled_adapters(name=adapter_name, enabled=True)  # enable only the current adapter we want to train

[NeMo I 2024-07-16 06:21:23 adapter_mixins:719] Setting adapter 'AN4' status : Enabled = False
[NeMo I 2024-07-16 06:21:23 adapter_mixins:734] Setting adapter 'AN4' status : Enabled = True


In [55]:
model.freeze()
model.unfreeze_enabled_adapters()
#model.unfreeze()
model.decoder.unfreeze()

[NeMo I 2024-07-16 06:21:23 adapter_mixins:405] Froze module encoder.layers.0.conv.batch_norm: BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
[NeMo I 2024-07-16 06:21:23 adapter_mixins:405] Froze module encoder.layers.1.conv.batch_norm: BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
[NeMo I 2024-07-16 06:21:23 adapter_mixins:405] Froze module encoder.layers.2.conv.batch_norm: BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
[NeMo I 2024-07-16 06:21:23 adapter_mixins:405] Froze module encoder.layers.3.conv.batch_norm: BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
[NeMo I 2024-07-16 06:21:23 adapter_mixins:405] Froze module encoder.layers.4.conv.batch_norm: BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
[NeMo I 2024-07-16 06:21:23 adapter_mixins:405] Froze module encoder.layers.5.conv.batch_norm: BatchNorm1d(512, eps

In [56]:
model.summarize()

  | Name              | Type                              | Params
------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0     
1 | encoder           | ConformerEncoderAdapter           | 122 M 
2 | spec_augmentation | SpectrogramAugmentation           | 0     
3 | wer               | WER                               | 0     
4 | decoder           | ConvASRDecoder                    | 619 K 
5 | loss              | CTCLoss                           | 0     
------------------------------------------------------------------------
1.8 M     Trainable params
121 M     Non-trainable params
123 M     Total params
493.010   Total estimated model params size (MB)

In [57]:
# Prepare NeMo's Experiment manager to handle checkpoint saving and logging for us
from nemo.utils import exp_manager

# Environment variable generally used for multi-node multi-gpu training.
# In notebook environments, this flag is unnecessary and can cause logs of multiple training runs to overwrite each other.
os.environ.pop('NEMO_EXPM_VERSION', None)

exp_config = exp_manager.ExpManagerConfig(
    exp_dir=f'/external2/karan_exp/experiments/',
    name=f"encoder:fixed-adapter64-decode:tune-multidomain-v4",
    checkpoint_callback_params=exp_manager.CallbackParams(
        monitor="val_wer",
        mode="min",
        always_save_nemo=True,
        save_best_model=True,
    ),
)

exp_config = OmegaConf.structured(exp_config)

logdir = exp_manager.exp_manager(trainer, exp_config)

[NeMo I 2024-07-16 06:21:24 exp_manager:396] Experiments will be logged at /external2/karan_exp/experiments/encoder:fixed-adapter64-decode:tune-multidomain-v4/2024-07-16_06-21-24
[NeMo I 2024-07-16 06:21:24 exp_manager:842] TensorboardLogger has been set up


[NeMo W 2024-07-16 06:21:24 exp_manager:952] The checkpoint callback was told to monitor a validation value and trainer's max_steps was set to 30000. Please ensure that max_steps will run for at least 1 epochs to ensure that checkpointing will not error out.


In [58]:
model.cfg['tokenizer']



{'dir': PosixPath('/external2/models/hf/stt_en_conformer_ctc_large/new_tokenizer'), 'type': 'bpe', 'model_path': '/external2/models/hf/stt_en_conformer_ctc_large/new_tokenizer/tokenizer.model', 'vocab_path': '/external2/models/hf/stt_en_conformer_ctc_large/new_tokenizer/vocab.txt', 'spe_tokenizer_vocab': '/external2/models/hf/stt_en_conformer_ctc_large/new_tokenizer/tokenizer.vocab'}

In [59]:
model.summarize()

  | Name              | Type                              | Params
------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0     
1 | encoder           | ConformerEncoderAdapter           | 122 M 
2 | spec_augmentation | SpectrogramAugmentation           | 0     
3 | wer               | WER                               | 0     
4 | decoder           | ConvASRDecoder                    | 619 K 
5 | loss              | CTCLoss                           | 0     
------------------------------------------------------------------------
1.8 M     Trainable params
121 M     Non-trainable params
123 M     Total params
493.010   Total estimated model params size (MB)

In [60]:
# Finally, train the adapters
trainer.fit(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


[NeMo I 2024-07-16 06:21:24 modelPT:723] Optimizer config = AdamW (
    Parameter Group 0
        amsgrad: False
        betas: [0.9, 0.98]
        capturable: False
        differentiable: False
        eps: 1e-08
        foreach: None
        fused: None
        lr: 0.1
        maximize: False
        weight_decay: 0.0
    )
[NeMo I 2024-07-16 06:21:24 lr_scheduler:915] Scheduler "<nemo.core.optim.lr_scheduler.NoamAnnealing object at 0x7c3af09af8b0>" 
    will be used during training (effective maximum steps = 30000) - 
    Parameters : 
    (d_model: 512
    warmup_steps: 100
    warmup_ratio: null
    min_lr: 1.0e-06
    max_steps: 30000
    )



  | Name              | Type                              | Params
------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0     
1 | encoder           | ConformerEncoderAdapter           | 122 M 
2 | spec_augmentation | SpectrogramAugmentation           | 0     
3 | wer               | WER                               | 0     
4 | decoder           | ConvASRDecoder                    | 619 K 
5 | loss              | CTCLoss                           | 0     
------------------------------------------------------------------------
1.8 M     Trainable params
121 M     Non-trainable params
123 M     Total params
493.010   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

[NeMo I 2024-07-16 06:21:25 wer:318] 
    
[NeMo I 2024-07-16 06:21:25 wer:319] reference:T3-T826-T248 T1_T248_T668 i T0 T1_T201 am experiencing T0 T1_T667 shortness of breath T0 T1_T54 lately T0 T2_T667_T47
[NeMo I 2024-07-16 06:21:25 wer:320] predicted:ent willT65T912T51T161T666 tcT161T94T1017T517T600T139T161T865T477ri tT867T161T1014T820T73T820T616T161T1006]T841T154T161T42T987T161T348T355T161T93T161T983T161T376
[NeMo I 2024-07-16 06:21:25 wer:318] 
    
[NeMo I 2024-07-16 06:21:25 wer:319] reference:T3-T826-T248 T1_T248_T668 i T0 T1_T201 was diagnosed with T0 T1_T12 fibromyalgia T0 T1_T623 three years ago T0 T2_T826_T689
[NeMo I 2024-07-16 06:21:25 wer:320] predicted:entT376T3T161T1021T161edorT635T883T841T137terT354T40T161T31T39T831T161T457T161T133T1022wT303T346T161T258T529T1001T146T161laT1021entT452T619T146 t moT754T368T161T501T346T376


Training: 0it [00:00, ?it/s]

[NeMo I 2024-07-16 06:21:26 preemption:56] Preemption requires torch distributed to be initialized, disabling preemption
[NeMo I 2024-07-16 06:21:53 wer:318] 
    
[NeMo I 2024-07-16 06:21:53 wer:319] reference:T3-T510-T511 T1_T201 change T0 T1_T988_T421 flight T0 to T1_T387 rome T0 on T1_T623 april 20th T0 T2_T334_T988
[NeMo I 2024-07-16 06:21:53 wer:320] predicted:_T649
